## Class Weights

En este dataset usaremos técnicas de pesos en las clases con menos representación para analizar si así logramos mejorar el recall de las enfermedades.

Este notebook será una antesala al modelo final

In [1]:
!pip install torchmetrics

# 1. Instalar la API de Kaggle
!pip install -q kaggle

# 2. Subir kaggle.json
from google.colab import files
files.upload()  # Subí aquí el archivo kaggle.json

# 3. Configurar kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. Descargar dataset y modelo desde Kaggle
!kaggle datasets download -d abdallahalidev/plantvillage-dataset
!unzip -q plantvillage-dataset.zip -d plantvillage_dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
100% 2.03G/2.04G [00:13<00:00, 230MB/s]
100% 2.04G/2.04G [00:13<00:00, 166MB/s]


In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from torch.amp import autocast, GradScaler
from tqdm import tqdm

import torch
import torchmetrics
import cv2 as cv
import torch.nn as nn

from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
# Species mapping
species_es_map = {
    'Strawberry': 'Fresa',
    'Grape': 'Uva',
    'Potato': 'Papa',
    'Blueberry': 'Arándano',
    'Corn_(maize)': 'Maíz',
    'Tomato': 'Tomate',
    'Peach': 'Durazno',
    'Pepper,_bell': 'Pimiento',
    'Orange': 'Naranja',
    'Cherry_(including_sour)': 'Cereza',
    'Apple': 'Manzana',
    'Raspberry': 'Frambuesa',
    'Squash': 'Calabaza',
    'Soybean': 'Soja'
}

# Disease mapping
disease_es_map = {
    'Black_rot': 'Podredumbre negra',
    'Early_blight': 'Tizón temprano',
    'Target_Spot': 'Mancha diana',
    'Late_blight': 'Tizón tardío',
    'Tomato_mosaic_virus': 'Virus del mosaico',
    'Haunglongbing_(Citrus_greening)': 'Huanglongbing (enverdecimiento de los cítricos)',
    'Leaf_Mold': 'Moho de la hoja',
    'Leaf_blight_(Isariopsis_Leaf_Spot)': 'Tizón de la hoja',
    'Powdery_mildew': 'Oídio',
    'Cedar_apple_rust': 'Roya del manzano y cedro',
    'Bacterial_spot': 'Mancha bacteriana',
    'Common_rust_': 'Roya común',
    'Esca_(Black_Measles)': 'Esca',
    'Tomato_Yellow_Leaf_Curl_Virus': 'Virus del rizado amarillo de la hoja',
    'Apple_scab': 'Sarna',
    'Northern_Leaf_Blight': 'Tizón foliar del norte',
    'Spider_mites Two-spotted_spider_mite': 'Ácaros araña de dos manchas',
    'Septoria_leaf_spot': 'Mancha foliar por septoria',
    'Cercospora_leaf_spot Gray_leaf_spot': 'Mancha foliar por cercospora / mancha foliar gris',
    'Leaf_scorch': 'Quemadura de la hoja'
}

In [4]:
# Directorio en Kaggle
dataset_path = "plantvillage_dataset/plantvillage dataset/color"
# Directorio para corrida local
# dataset_path = "../data/plantvillage/plantvillage dataset/color"

data = []

# Recorremos carpetas y archivos
if os.path.exists(dataset_path):
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if not os.path.isdir(folder_path):
            continue
        species, disease = folder.split('___', 1)
        healthy = disease == 'healthy'
        disease = None if healthy else disease
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            if os.path.isfile(file_path):
                data.append({
                    'Format': 'color',
                    'Species': species,
                    'Healthy': healthy,
                    'Disease': disease,
                    'FileName': file
                })

# Crear DataFrame
df = pd.DataFrame(data)

In [5]:
# Agregar nombre en español para especie
df['Especie'] = df['Species'].map(species_es_map)

# Agregar nombre en español para enfermedad (o 'Sano' si es healthy)
df['Enfermedad'] = df.apply(
    lambda row: 'Sano' if row['Healthy'] else disease_es_map.get(row['Disease'], row['Disease']),
    axis=1
)

# Duplicar columna "Enfermedad" como "Label", por cuestiones prácticas
df['Label'] = df['Enfermedad']

# Codificar las etiquetas compuestas como números
df['Label_id'] = df['Label'].astype('category').cat.codes

# Crear el diccionario de mapeo id → etiqueta compuesta
label_map = dict(enumerate(df['Label'].astype('category').cat.categories))

# Número de clases únicas
NUM_CLASSES = len(label_map)

In [6]:
# Semilla reproducible
SEED = 42

# Split 85% train, 15% valid
train_df, valid_df = train_test_split(
    df,
    test_size=0.15,
    stratify=df['Label_id'],
    random_state=SEED
)

In [7]:
class PlantVillageDataset(torch.utils.data.Dataset):
    def __init__(self, df, root_dir, format_type='color', transform=None):
        self.df = df.reset_index(drop=True)
        self.root_dir = root_dir
        self.format_type = format_type
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        folder = f"{row['Species']}___{'healthy' if row['Healthy'] else row['Disease']}"
        image_path = os.path.join(self.root_dir, folder, row['FileName'])

        image = cv.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {image_path}")

        # Convertir BGR a RGB
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        # image es numpy array uint8, listo para ToPILImage
        label = torch.tensor(row['Label_id'], dtype=torch.long)

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
# Verificamos si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [11]:
IMAGE_SIZE = (224, 224)

# Normalización estándar ImageNet
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

In [13]:
train_dataset = PlantVillageDataset(train_df, dataset_path, transform=train_transform)
valid_dataset = PlantVillageDataset(valid_df, dataset_path, transform=val_transform)

BATCH_SIZE = 128

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [14]:
# Cargamos el modelo sin pesos preentrenados
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Congelar todos los parámetros para no reentrenar
for param in model.parameters():
    param.requires_grad = False

# Modificamos la capa final según tu número de clases
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

# Movemos el modelo al dispositivo (GPU o CPU)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 139MB/s]


In [17]:
from sklearn.utils.class_weight import compute_class_weight


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

all_train_labels = train_dataset.df['Label_id'].values

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_train_labels),
    y=all_train_labels
)

# Convertimos a tensor y movemos a GPU si es necesario
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Pérdida ponderada
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Métrica: precisión
metric = torchmetrics.classification.MulticlassAccuracy(num_classes=NUM_CLASSES, average='macro')
metric.to(device)

MulticlassAccuracy()

In [18]:
# Info de loaders e imagen
data_dict = {
    "train": train_loader,
    "valid": valid_loader,
    "image_width": 224,
    "image_height": 224
}

# TensorBoard
writer = {
    "train": SummaryWriter(log_dir="runs/plant_train"),
    "valid": SummaryWriter(log_dir="runs/plant_valid")
}

In [19]:
def train(model, optimizer, criterion, metric, data, epochs, tb_writer=None, log_interval=10, early_stop_patience=3):
    """
    Entrena un modelo de clasificación utilizando PyTorch con validación,
    AMP (Automatic Mixed Precision), early stopping y registro en TensorBoard.

    Args:
        model (nn.Module): Modelo de PyTorch a entrenar.
        optimizer (torch.optim.Optimizer): Optimizador utilizado (ej. Adam, SGD).
        criterion (nn.Module): Función de pérdida (ej. CrossEntropyLoss).
        metric (torchmetrics.Metric): Métrica de evaluación (ej. Accuracy).
        data (dict): Diccionario con dataloaders y dimensiones:
            - 'train': DataLoader de entrenamiento.
            - 'valid': DataLoader de validación.
            - 'image_width': Ancho de las imágenes.
            - 'image_height': Alto de las imágenes.
        epochs (int): Cantidad máxima de épocas para entrenar.
        tb_writer (dict, opcional): Diccionario con SummaryWriter de TensorBoard para 'train' y 'valid'.
        log_interval (int, opcional): Frecuencia (en batches) con la que se muestra el progreso del entrenamiento. Default: 10.
        early_stop_patience (int, opcional): Número de épocas sin mejora en validación antes de detener entrenamiento. Default: 3.

    Returns:
        dict: Diccionario con el historial de métricas:
            - 'train_loss': Lista de pérdidas por época en entrenamiento.
            - 'train_acc': Lista de accuracies por época en entrenamiento.
            - 'valid_loss': Lista de pérdidas por época en validación.
            - 'valid_acc': Lista de accuracies por época en validación.

    Efectos secundarios:
        - Guarda el mejor modelo (según pérdida de validación más baja) en el archivo 'mejor_modelo.pth'.
        - Muestra progreso en consola con tqdm.
        - Escribe métricas en TensorBoard si se especifica tb_writer.
    """
    train_loader = data["train"]
    valid_loader = data["valid"]
    image_width, image_height = data["image_width"], data["image_height"]

    device = next(model.parameters()).device
    scaler = GradScaler(device='cuda')

    train_writer = tb_writer["train"] if tb_writer else None
    valid_writer = tb_writer["valid"] if tb_writer else None

    if tb_writer:
        dummy_input = torch.zeros((1, 3, image_width, image_height)).to(device)
        train_writer.add_graph(model, dummy_input)

    best_val_loss = float("inf")
    patience_counter = 0

    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []

    for epoch in range(epochs):
        model.train()
        metric.reset()
        epoch_train_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs} [Train]")

        for batch_idx, (train_data, train_target) in progress_bar:
            train_data, train_target = train_data.to(device), train_target.to(device)

            optimizer.zero_grad()
            with autocast(device_type='cuda'):
                output = model(train_data)
                loss = criterion(output, train_target)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            acc = metric(output, train_target)
            epoch_train_loss += loss.item()

            if batch_idx % log_interval == 0:
                avg_loss = epoch_train_loss / (batch_idx + 1)
                avg_acc = acc.item()
                progress_bar.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{avg_acc:.4f}")

        train_accuracy = metric.compute().item()
        train_loss.append(epoch_train_loss / len(train_loader))
        train_acc.append(train_accuracy)

        # Validación
        model.eval()
        metric.reset()
        val_loss_total = 0.0
        with torch.no_grad():
            for val_data, val_target in valid_loader:
                val_data, val_target = val_data.to(device), val_target.to(device)
                with autocast(device_type='cuda'):
                    output = model(val_data)
                    loss = criterion(output, val_target)
                metric.update(output, val_target)
                val_loss_total += loss.item()

        val_accuracy = metric.compute().item()
        current_val_loss = val_loss_total / len(valid_loader)
        valid_loss.append(current_val_loss)
        valid_acc.append(val_accuracy)

        print(f"Epoch {epoch+1} - Train Loss: {train_loss[-1]:.4f} Acc: {train_accuracy:.4f} | "
              f"Valid Loss: {current_val_loss:.4f} Acc: {val_accuracy:.4f}")

        if tb_writer:
            train_writer.add_scalar("loss", train_loss[-1], epoch)
            valid_writer.add_scalar("loss", valid_loss[-1], epoch)
            train_writer.add_scalar("accuracy", train_accuracy, epoch)
            valid_writer.add_scalar("accuracy", val_accuracy, epoch)
            train_writer.flush()
            valid_writer.flush()

        # Early stopping y guardado
        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
            torch.save(best_model_state, "mejor_modelo.pth")
            print("✅ Mejor modelo guardado como 'mejor_modelo.pth'")
        else:
            patience_counter += 1
            if patience_counter >= early_stop_patience:
                print(f"⏹️ Early stopping at epoch {epoch+1}")
                break

    return {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "valid_loss": valid_loss,
        "valid_acc": valid_acc
    }


In [20]:
# Entrenamiento
history = train(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    metric=metric,
    data=data_dict,
    epochs=50,
    tb_writer=writer
)

Epoch 1/50 [Train]: 100%|██████████| 361/361 [03:04<00:00,  1.95it/s, acc=0.6339, loss=2.4371]


Epoch 1 - Train Loss: 2.4371 Acc: 0.3869 | Valid Loss: 1.8784 Acc: 0.5987
✅ Mejor modelo guardado como 'mejor_modelo.pth'


Epoch 2/50 [Train]: 100%|██████████| 361/361 [02:51<00:00,  2.11it/s, acc=0.5835, loss=1.5751]


Epoch 2 - Train Loss: 1.5751 Acc: 0.7077 | Valid Loss: 1.3192 Acc: 0.7446
✅ Mejor modelo guardado como 'mejor_modelo.pth'


Epoch 3/50 [Train]: 100%|██████████| 361/361 [02:47<00:00,  2.16it/s, acc=0.6355, loss=1.1709]


...

Epoch 48 - Train Loss: 0.2610 Acc: 0.9197 | Valid Loss: 0.2806 Acc: 0.9100


Epoch 49/50 [Train]: 100%|██████████| 361/361 [02:47<00:00,  2.16it/s, acc=0.8870, loss=0.2579]


Epoch 49 - Train Loss: 0.2579 Acc: 0.9219 | Valid Loss: 0.2768 Acc: 0.9106


Epoch 50/50 [Train]: 100%|██████████| 361/361 [02:45<00:00,  2.18it/s, acc=0.8804, loss=0.2601]


Epoch 50 - Train Loss: 0.2601 Acc: 0.9201 | Valid Loss: 0.2757 Acc: 0.9100
⏹️ Early stopping at epoch 50


In [21]:
def plot_training_curves(history, output_dir="training_plots"):
    os.makedirs(output_dir, exist_ok=True)

    # Curva de pérdida
    plt.figure(figsize=(8, 6))
    plt.plot(history['train_loss'], label='Entrenamiento', marker='o')
    plt.plot(history['valid_loss'], label='Validación', marker='o')
    plt.title("Pérdida durante el entrenamiento")
    plt.xlabel("Época")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, "loss_curve.png"))
    plt.close()

    # Curva de accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(history['train_acc'], label='Entrenamiento', marker='o')
    plt.plot(history['valid_acc'], label='Validación', marker='o')
    plt.title("Precisión durante el entrenamiento")
    plt.xlabel("Época")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, "accuracy_curve.png"))
    plt.close()

    print(f"✅ Gráficos guardados en {output_dir}/")

# Usar después de entrenar
plot_training_curves(history)

✅ Gráficos guardados en training_plots/


In [22]:
correct = 0
total = 0
model.eval()

all_preds = []
all_labels = []

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=False
)

with torch.no_grad():
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [23]:
label_map = dict(enumerate(df['Label'].astype('category').cat.categories))

In [25]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(label_map.values()),
            yticklabels=list(label_map.values()))
plt.xlabel('Predicción')
plt.ylabel('Etiqueta real')
plt.title('Matriz de Confusión')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("matriz_confusion.png")
plt.close()
print("✅ Matriz de confusión guardada como 'matriz_confusion.png'")

✅ Matriz de confusión guardada como 'matriz_confusion.png'


In [26]:
from sklearn.metrics import classification_report, accuracy_score

# Accuracy global
acc = accuracy_score(all_labels, all_preds)
print(f"Accuracy global: {acc:.4f}")

# Reporte detallado por clase (precision, recall, f1-score, support)
report = classification_report(
    all_labels, all_preds,
    target_names=list(label_map.values()),
    digits=4,
    output_dict=True
)

# Imprimir reporte legible
import pandas as pd
df_report = pd.DataFrame(report).transpose()
print("\n=== Reporte de clasificación por clase ===")
print(df_report)

# Guardar reporte a CSV para análisis posterior
df_report.to_csv("reporte_clasificacion.csv")
print("✅ Reporte de clasificación guardado como 'reporte_clasificacion.csv'")

Accuracy global: 0.9195

=== Reporte de clasificación por clase ===
                                                   precision    recall  \
Esca                                                0.975124  0.946860   
Huanglongbing (enverdecimiento de los cítricos)     0.978469  0.990315   
Mancha bacteriana                                   0.884298  0.921279   
Mancha diana                                        0.659574  0.881517   
Mancha foliar por cercospora / mancha foliar gris   0.666667  0.831169   
Mancha foliar por septoria                          0.827957  0.868421   
Moho de la hoja                                     0.798742  0.888112   
Oídio                                               0.970149  0.900693   
Podredumbre negra                                   0.846906  0.962963   
Quemadura de la hoja                                0.987730  0.969880   
Roya común                                          0.983051  0.972067   
Roya del manzano y cedro                    

In [28]:
df_report

,precision,recall,f1-score,support
Esca,0.975124,0.946860,0.960784,207.00000
Huanglongbing (enverdecimiento de los cítricos),0.978469,0.990315,0.984356,826.00000
Mancha bacteriana,0.884298,0.921279,0.902410,813.00000
Mancha diana,0.659574,0.881517,0.754564,211.00000
Mancha foliar por cercospora / mancha foliar gris,0.666667,0.831169,0.739884,77.00000
Mancha foliar por septoria,0.827957,0.868421,0.847706,266.00000
Moho de la hoja,0.798742,0.888112,0.841060,143.00000
Oídio,0.970149,0.900693,0.934132,433.00000
Podredumbre negra,0.846906,0.962963,0.901213,270.00000
Quemadura de la hoja,0.987730,0.969880,0.978723,166.00000


In [31]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Índice de la clase 'Sano'
sano_idx = None
for k, v in label_map.items():
    if v.lower() == 'sano':
        sano_idx = k
        break
if sano_idx is None:
    raise ValueError("No se encontró la clase 'Sano' en label_map")

all_labels_np = np.array(all_labels)
all_preds_np = np.array(all_preds)

# Etiquetas binarias: 1 = enfermedad, 0 = sano
labels_bin = (all_labels_np != sano_idx).astype(int)
preds_bin = (all_preds_np != sano_idx).astype(int)

# Métricas para enfermedades (clase positiva = enfermedad)
precision_enf = precision_score(labels_bin, preds_bin)
recall_enf = recall_score(labels_bin, preds_bin)
f1_enf = f1_score(labels_bin, preds_bin)

# Métricas para sano (clase positiva = sano)
labels_bin_sano = (all_labels_np == sano_idx).astype(int)
preds_bin_sano = (all_preds_np == sano_idx).astype(int)

precision_sano = precision_score(labels_bin_sano, preds_bin_sano)
recall_sano = recall_score(labels_bin_sano, preds_bin_sano)
f1_sano = f1_score(labels_bin_sano, preds_bin_sano)

print("Métricas para enfermedades (todas combinadas vs sano):")
print(f"Precision: {precision_enf:.4f}")
print(f"Recall:    {recall_enf:.4f}")
print(f"F1-score:  {f1_enf:.4f}\n")

print("Métricas para sano (vs todas las enfermedades):")
print(f"Precision: {precision_sano:.4f}")
print(f"Recall:    {recall_sano:.4f}")
print(f"F1-score:  {f1_sano:.4f}")


Métricas para enfermedades (todas combinadas vs sano):
Precision: 0.9700
Recall:    0.9910
F1-score:  0.9804

Métricas para sano (vs todas las enfermedades):
Precision: 0.9752
Recall:    0.9205
F1-score:  0.9470


Observamos una mejora en el recall general de las especies enfermas, reduciendo la cantidad de casos donde se clasifican como sanas o se clasifican con una enfermedad incorrecta. Esto es especialmente importante en aplicaciones agrícolas donde detectar enfermedades a tiempo puede prevenir pérdidas significativas.

A costa de ello, hemos empeorado el recall de la clase 'Sano'. Esto puede devenir en tratamientos innecesarios, los cuales pueden ser costosos.

En el trade-off, el resultado es positivo, dado que vamos a priorizar no equivocarnos al clasificar una planta enferma como sana, pero vamos a intentar mejorar el modelo en general en la próxima iteración.